In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 8.3 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
     |████████████████████████████████| 6.6 MB 55.4 MB/s 
     |████████████████████████████████| 101 kB 13.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import json
import pandas as pd
from textwrap import wrap

In [ ]:
def json_to_pandas1(json_corpus):
  with open(json_corpus) as f:
    corpus = json.load(f)

  subcategories = corpus.keys()

  categories_column = []
  subcategories_column = []
  paragraphs_column = []

  for subcategory in subcategories:
    cat_dict = corpus[subcategory]
    category = cat_dict["category"]
    paragraphs = cat_dict["true"]
    categories_column.extend([category]*len(paragraphs))
    subcategories_column.extend([subcategory]*len(paragraphs))
    for paragraph in paragraphs:
      paragraphs_column.append(paragraph["paragraph"])

  return pd.DataFrame({'paragraph': paragraphs_column, 'category': categories_column, 'subcategory': subcategories_column})

In [ ]:
def json_to_pandas2(json_corpus):
  with open(json_corpus) as f:
    corpus = json.load(f)

  categories_column = []
  subcategories_column = []
  paragraphs_column = []

  for dictionary in corpus:
    category = dictionary["Categoría"].strip()
    subcategory = dictionary["Subcategoría"].strip()
    content = dictionary["content"].split('\n')
    categories_column.extend([category]*len(content))
    subcategories_column.extend([subcategory]*len(content))
    for paragraph in content:
      paragraphs_column.append(paragraph)

  return pd.DataFrame({'paragraph': paragraphs_column, 'category': categories_column, 'subcategory': subcategories_column})

In [ ]:
# Load and concatenate datasets
DATASET1_PATH = '/content/drive/My Drive/Naveler/03.TECH/data_discovery/datasets/tf_idf_data_corpus_data_corpus.json'
DATASET2_PATH = '/content/drive/My Drive/Naveler/03.TECH/data_discovery/datasets/tf_idf_data_output_wiki_content.json'

from google.colab import drive
drive.mount('/content/drive')

df1 = json_to_pandas1(DATASET1_PATH)
df2 = json_to_pandas2(DATASET2_PATH)
df = pd.concat([df1, df2], ignore_index=True)

Mounted at /content/drive


In [ ]:
# Initial parameters
ind = 1 # set to 0 to classify by categories, set to 1 to classify by subcategories
CLASSIFY_BY = ['category', 'subcategory'][ind]
RANDOM_SEED = 42
MAX_LEN = 250
test_prop = 0.2 # proportion of test data
BATCH_SIZE = 16 # training batches size

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
# Check data
print("Number of samples: {}".format(df.shape[0]))
ind = 10000
print("\nSample example:")
print("\n".join(wrap(df['paragraph'][ind])))
print("Category: {}".format(df['category'][ind]))
print("Subcategory: {}".format(df['subcategory'][ind]))

Number of samples: 29053

Sample example:
Los redespachos por restricciones del PDBF se comprobarán horariamente
mientras no existan productos cuarto-horarios en los mercados diario e
intradiario y, por tanto, de BOLETÍN OFICIAL DEL ESTADO Martes 29 de
marzo de 2022 Sec. III. Pág. 41336 acuerdo con lo dispuesto en el PO
3.2. Los valores de energía y precio en todos los cuartos de hora de
la misma hora tengan el mismo valor.
Category: Sector eléctrico
Subcategory: Consumo eléctrico


In [ ]:
# Code categories
if CLASSIFY_BY == 'category':
  possible_labels = df.category.unique()
elif CLASSIFY_BY == 'subcategory':
  possible_labels = df.subcategory.unique()
NCLASSES = len(possible_labels)
print("Number of classes: {}\n".format(NCLASSES))

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

Number of classes: 12



{'Bebidas energéticas': 11,
 'Calidad aire': 0,
 'Cambio climático': 1,
 'Consumo eléctrico': 7,
 'Eficiencia energética': 3,
 'GNL maritimo': 4,
 'Gestión del agua': 2,
 'Movilidad sostenible': 5,
 'Redes eléctricas': 8,
 'Smart Grids': 6,
 'Tarifas  / mediciones eléctricas': 9,
 'Vulnerabilidad energética': 10}

In [ ]:
# Add label column
if CLASSIFY_BY == 'category':
  df['label'] = df.category.replace(label_dict)
elif CLASSIFY_BY == 'subcategory':
  df['label'] = df.subcategory.replace(label_dict)
df.head()

,paragraph,category,subcategory,label
0,RELATIVA A REVISAR LA ACTUAL CLASIFICACIÓN DE ...,Impacto energético,Calidad aire,0
1,b) Las actividades en el marco del régimen de ...,Impacto energético,Calidad aire,0
2,c) La compensación de los costes indirectos de...,Impacto energético,Calidad aire,0
3,d) Las actividades relacionadas con vertederos...,Impacto energético,Calidad aire,0
4,e) Las actividades en las que la eliminación a...,Impacto energético,Calidad aire,0


In [ ]:
df[CLASSIFY_BY].value_counts()

Redes eléctricas                    7955
Consumo eléctrico                   5963
Tarifas  / mediciones eléctricas    5563
Calidad aire                        2059
Eficiencia energética               1829
Gestión del agua                    1563
Vulnerabilidad energética           1453
Cambio climático                    1258
Smart Grids                         1042
GNL maritimo                         208
Bebidas energéticas                  121
Movilidad sostenible                  39
Name: subcategory, dtype: int64

In [ ]:
# Define tokenizer
PRE_TRAINED_MODEL_NAME = 'dccuchile/bert-base-spanish-wwm-cased'
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

Downloading:   0%|          | 0.00/236k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/134 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/364 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/648 [00:00<?, ?B/s]

In [ ]:
# Tokenization example
sample_txt = 'Las bebidas azucaradas son aquellas que contienen azúcar añadido.'
tokens = tokenizer.tokenize(sample_txt)
token_ids = tokenizer.convert_tokens_to_ids(tokens)
print('Frase: ', sample_txt)
print('Tokens: ', tokens)
print('Tokens numéricos: ', token_ids)

Frase:  Las bebidas azucaradas son aquellas que contienen azúcar añadido.
Tokens:  ['Las', 'bebidas', 'azucar', '##adas', 'son', 'aquellas', 'que', 'contienen', 'azúcar', 'añadido', '.']
Tokens numéricos:  [1613, 12779, 28787, 1319, 1404, 8587, 1038, 11730, 7585, 12877, 1009]


In [ ]:
# Codification for BERT (data pre-processing)
encoding = tokenizer.encode_plus(
    sample_txt,
    max_length = MAX_LEN,
    truncation = True,
    add_special_tokens = True,
    return_token_type_ids = False,
    #pad_to_max_length = True,
    padding = 'max_length',
    return_attention_mask = True,
    return_tensors = 'pt'
)

In [ ]:
encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [ ]:
print(tokenizer.convert_ids_to_tokens(encoding['input_ids'][0]))
print(encoding['input_ids'][0])
print(encoding['attention_mask'][0])

['[CLS]', 'Las', 'bebidas', 'azucar', '##adas', 'son', 'aquellas', 'que', 'contienen', 'azúcar', 'añadido', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD

In [ ]:
# Define class for dataset
class BOEDataset(Dataset):

  def __init__(self, paragraphs, labels, tokenizer, max_len):
    self.paragraphs = paragraphs
    self.labels = labels
    self.tokenizer = tokenizer
    self.max_len = max_len

  def __len__(self):
      return len(self.paragraphs)
    
  def __getitem__(self, item):
    paragraph = str(self.paragraphs[item])
    label = self.labels[item]
    encoding = tokenizer.encode_plus(
        paragraph,
        max_length = self.max_len,
        truncation = True,
        add_special_tokens = True,
        return_token_type_ids = False,
        #pad_to_max_length = True,
        padding = 'max_length',
        return_attention_mask = True,
        return_tensors = 'pt'
        )
    return {
          'paragraph': paragraph,
          'input_ids': encoding['input_ids'].flatten(),
          'attention_mask': encoding['attention_mask'].flatten(),
          'label': torch.tensor(label, dtype=torch.long)
      }

In [ ]:
# Define data loader
def data_loader(df, tokenizer, max_len, batch_size):
  dataset = BOEDataset(
      paragraphs = df.paragraph.to_numpy(),
      labels = df.label.to_numpy(),
      tokenizer = tokenizer,
      max_len = MAX_LEN
  )
  return DataLoader(dataset, batch_size = BATCH_SIZE, num_workers = 2)

In [ ]:
# Split train and test data
df_train, df_test = train_test_split(df, test_size = test_prop, random_state=RANDOM_SEED, stratify=df.label.values)

train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [ ]:
df_train[CLASSIFY_BY].value_counts()

Redes eléctricas                    6364
Consumo eléctrico                   4770
Tarifas  / mediciones eléctricas    4450
Calidad aire                        1647
Eficiencia energética               1463
Gestión del agua                    1250
Vulnerabilidad energética           1162
Cambio climático                    1007
Smart Grids                          834
GNL maritimo                         167
Bebidas energéticas                   97
Movilidad sostenible                  31
Name: subcategory, dtype: int64

In [ ]:
df_test[CLASSIFY_BY].value_counts()

Redes eléctricas                    1591
Consumo eléctrico                   1193
Tarifas  / mediciones eléctricas    1113
Calidad aire                         412
Eficiencia energética                366
Gestión del agua                     313
Vulnerabilidad energética            291
Cambio climático                     251
Smart Grids                          208
GNL maritimo                          41
Bebidas energéticas                   24
Movilidad sostenible                   8
Name: subcategory, dtype: int64

In [ ]:
# Define class for building model
class BERTTextClassifier(nn.Module):

  def __init__(self, n_classes):
    super(BERTTextClassifier, self).__init__()
    self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME, return_dict=False)
    self.drop = nn.Dropout(p=0.3)
    self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

  def forward(self, input_ids, attention_mask):
    _, cls_output = self.bert(
        input_ids = input_ids,
        attention_mask = attention_mask
    )
    drop_output = self.drop(cls_output)
    output = self.linear(drop_output)
    return output

In [ ]:
# Build model (download)
model = BERTTextClassifier(NCLASSES)
model = model.to(device)

Downloading:   0%|          | 0.00/419M [00:00<?, ?B/s]

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.we

In [ ]:
# Check model
#print(model)
# BERT -> 31002 words codified into 768 values
# dropout layer
# linear layer (768xNCLASSES)

In [ ]:
# Training parameters
EPOCHS = 5
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_data_loader) * EPOCHS # (8000/16)*5
scheduler = get_linear_schedule_with_warmup( # gradually decrease learning rate (step length)
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)
loss_fn = nn.CrossEntropyLoss().to(device) # loss function to be minimized

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
# Training iteration
def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
  model = model.train()
  losses = []
  correct_predictions = 0
  for batch in data_loader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['label'].to(device)
    outputs = model(input_ids = input_ids, attention_mask = attention_mask)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)
    correct_predictions += torch.sum(preds == labels)
    losses.append(loss.item())
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
  return correct_predictions.double()/n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
  model = model.eval()
  losses = []
  correct_predictions = 0
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      outputs = model(input_ids = input_ids, attention_mask = attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)
      correct_predictions += torch.sum(preds == labels)
      losses.append(loss.item())
  return correct_predictions.double()/n_examples, np.mean(losses)

In [ ]:
# Training (high runtime!)
for epoch in range(EPOCHS):
  print('Epoch {} de {}'.format(epoch+1, EPOCHS))
  print('------------------')
  train_acc, train_loss = train_model(
      model, train_data_loader, loss_fn, optimizer, device, scheduler, len(df_train)
  )
  test_acc, test_loss = eval_model(
      model, test_data_loader, loss_fn, device, len(df_test)
  )
  print('Entrenamiento: Loss: {}, accuracy: {}'.format(train_loss, train_acc))
  print('Validación: Loss: {}, accuracy: {}'.format(test_loss, test_acc))
  print('')

Epoch 1 de 5
------------------
Entrenamiento: Loss: 1.1178202653497313, accuracy: 0.5035280956888392
Validación: Loss: 0.9202386546102199, accuracy: 0.5646188263637929

Epoch 2 de 5
------------------
Entrenamiento: Loss: 0.8029330333224347, accuracy: 0.5993029859736684
Validación: Loss: 0.8937645153357432, accuracy: 0.5783858199965582

Epoch 3 de 5
------------------
Entrenamiento: Loss: 0.6906388265382645, accuracy: 0.6379829618793563
Validación: Loss: 0.8986619867302559, accuracy: 0.5873343658578558

Epoch 4 de 5
------------------
Entrenamiento: Loss: 0.6352479389964181, accuracy: 0.6593666637982961
Validación: Loss: 0.9187406442322574, accuracy: 0.5842367922904836

Epoch 5 de 5
------------------
Entrenamiento: Loss: 0.6046727391579031, accuracy: 0.6705102831081662
Validación: Loss: 0.9392506843739814, accuracy: 0.5591120289106867



In [ ]:
# Model path
if CLASSIFY_BY == 'category':
  s = 'cat'
elif CLASSIFY_BY == 'subcategory':
  s = 'subcat'
MODEL_NAME = 'BERT_text_classifier_'+s
MODEL_PATH = '/content/drive/My Drive/Naveler/03.TECH/data_discovery/models/'+MODEL_NAME

In [ ]:
# Save model
torch.save(model.state_dict(), MODEL_PATH)

In [ ]:
# Upload model
loaded_model = BERTTextClassifier(NCLASSES)
loaded_model = loaded_model.to(device)
loaded_model.load_state_dict(torch.load(MODEL_PATH))
#model.eval()

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.we

<All keys matched successfully>

In [ ]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = preds.flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
def evaluate(model, data_loader):
  model = model.eval()
  predictions = []
  labels = []
  with torch.no_grad():
    for batch in data_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      label_ids = batch['label'].cpu().numpy()
      labels.append(label_ids)
      outputs = model(input_ids = input_ids, attention_mask = attention_mask)
      _, preds = torch.max(outputs, dim=1)
      preds = preds.detach().cpu().numpy()
      predictions.append(preds)

  predictions = np.concatenate(predictions, axis=0)
  labels = np.concatenate(labels, axis=0)
  return predictions, labels

In [ ]:
# Calculate accuracy per class on test data
predictions, labels = evaluate(loaded_model, test_data_loader)
accuracy_per_class(predictions, labels)

Class: Calidad aire
Accuracy: 369/412

Class: Cambio climático
Accuracy: 195/251

Class: Gestión del agua
Accuracy: 269/313

Class: Eficiencia energética
Accuracy: 322/366

Class: GNL maritimo
Accuracy: 28/41

Class: Movilidad sostenible
Accuracy: 2/8

Class: Smart Grids
Accuracy: 179/208

Class: Consumo eléctrico
Accuracy: 378/1193

Class: Redes eléctricas
Accuracy: 747/1591

Class: Tarifas  / mediciones eléctricas
Accuracy: 514/1113

Class: Vulnerabilidad energética
Accuracy: 234/291

Class: Bebidas energéticas
Accuracy: 12/24



In [ ]:
# Classify new data
def classifyParagraph(paragraph_text):
  encoding_paragraph = tokenizer.encode_plus(
      paragraph_text,
      max_length = MAX_LEN,
      truncation = True,
      add_special_tokens = True,
      return_token_type_ids = False,
      #pad_to_max_length = True,
      padding='max_length',
      return_attention_mask = True,
      return_tensors = 'pt'
      )
  
  input_ids = encoding_paragraph['input_ids'].to(device)
  attention_mask = encoding_paragraph['attention_mask'].to(device)
  output = loaded_model(input_ids, attention_mask)
  print("\n".join(wrap(paragraph_text)))
  with torch.no_grad():
    G = nn.functional.softmax(output, dim=1)[0]
  k = len(possible_labels)
  #print(G.topk(k))
  for (p, y) in zip(*(G.topk(k))):
    print(f"{possible_labels[y.item()]} ({100 * p.item():.2f}%)")

In [ ]:
# Classification example
paragraph = "Las bebidas azucaradas son perjudiciales para la salud, además tienen un impacto negativo para el medio ambiente."

classifyParagraph(paragraph)

Las bebidas azucaradas son perjudiciales para la salud, además tienen
un impacto negativo para el medio ambiente.
Bebidas energéticas (96.68%)
Eficiencia energética (1.07%)
GNL maritimo (0.83%)
Movilidad sostenible (0.70%)
Vulnerabilidad energética (0.20%)
Calidad aire (0.13%)
Gestión del agua (0.10%)
Cambio climático (0.09%)
Tarifas  / mediciones eléctricas (0.08%)
Smart Grids (0.06%)
Consumo eléctrico (0.04%)
Redes eléctricas (0.02%)
